<h6 style= 'color: blue'>  Часть вторая </h6>  


In [11]:

# Импортируем библиотеку numpy для работы с линейной алгеброй
import numpy as np
# Импортируем библиотеку pandas для работы с данными, чтения и записи файлов в формате CSV
import pandas as pd
# Импортируем модуль warnings для управления предупреждениями, которые могут возникать при выполнении кода
import warnings
# Используем метод filterwarnings для игнорирования всех предупреждений
warnings.filterwarnings("ignore")
# Импортируем модуль sklearn.model_selection для разделения данных на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split
# Импортируем модуль sklearn.metrics для оценки качества моделей машинного обучения
from sklearn.metrics import classification_report
# Импортируем модуль sklearn.preprocessing для стандартизации данных
from sklearn.preprocessing import StandardScaler
# Импортируем модуль matplotlib для работы с графикой
import matplotlib as plt




# Поиск оптимальных параметров для моделей

Посмотрим библиотеку Optuna, которая предназначена для автоматического подбора оптимальных гиперпараметров для различных моделей машинного обучения и библиотеку PyOD, ее основные функции, примеры использования и сравнение производительности различных моделей детекции выбросов.
Мы хотим найти оптимальные параметры для некоторых моделей детекции выбросов из библиотеки PyOD, чтобы позже построить их с оптимальными параметрами, проанализировать результаты и добиться максимальной интегральной точности с их помощью. Это интересная и сложная задача.



In [12]:

## Сначало загружаем данные

# Читаем данные из файла CSV и сохраняем их в датафрейме df
# Файл CSV содержит информацию о транзакциях с кредитных карт, которые были совершены в течение двух дней в сентябре 2013 года в Европе
# В файле есть 284807 строк и 31 столбец
df = pd.read_csv('creditcard.csv')
# Удаляем столбцы Time и Class из датафрейма и сохраняем оставшиеся столбцы в датафрейме X
# Столбец Time показывает время транзакции в секундах от первой транзакции в датасете, но он не влияет на то, является ли транзакция мошеннической или нет
# Столбец Class показывает метку класса для транзакции (0 - не мошенническая, 1 - мошенническая), но он не является признаком, а является целевой переменной, которую мы хотим предсказать
X = df.drop(columns=['Time', 'Class'])
# Сохраняем столбец Class в серии y, которая будет содержать метки классов для транзакций
y = df.Class


FileNotFoundError: ignored

In [ ]:

# Просматриваем
df.head(5)


In [ ]:

# Просматриваем
y.head (5)



In [ ]:

# Просматриваем
X.head(5)


In [ ]:

# Импортируем класс StandardScaler из модуля sklearn.preprocessing для стандартизации данных
from sklearn.preprocessing import StandardScaler
# Создаем объект norm, который будет представлять стандартизатор данных
norm = StandardScaler()
# Применяем метод fit_transform к датафрейму X, который содержит признаки транзакций, и получаем массив с нормализованными данными
# Метод fit_transform сначала вычисляет среднее и стандартное отклонение для каждого столбца в датафрейме, а затем преобразует каждое значение в столбце по формуле: (значение - среднее) / стандартное отклонение
# Таким образом, мы получаем данные, в которых каждый столбец имеет среднее равное нулю и стандартное отклонение равное единице
X_norm = pd.DataFrame(norm.fit_transform(X))



 Ниже, мы хотим определить функцию, которая возвращает часть данных о транзакциях с кредитных карт, которую можем использовать для обучения моделей детекции выбросов.


In [ ]:

# Определяем функцию get_data_part, которая принимает на вход датафрейм X с признаками транзакций, серию y с метками классов и параметр size, который задает долю данных, которую мы хотим взять
def get_data_part(X, y, size=0.1):
    # Импортируем функцию train_test_split из модуля sklearn.model_selection для разделения данных на обучающую и тестовую выборки
    from sklearn.model_selection import train_test_split
    # Применяем функцию train_test_split к датафрейму X и серии y, чтобы получить четыре объекта: X_train, X_test, y_train, y_test
    # Параметр random_state=0 означает, что мы фиксируем случайное состояние разделения для воспроизводимости результатов
    # Параметр train_size=size означает, что мы берем size процентов данных для обучающей выборки, а остальные данные для тестовой выборки
    # Параметр shuffle=True означает, что мы перемешиваем данные перед разделением, чтобы избежать смещения
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=0, train_size=size, shuffle=True
    )
    # Выводим на экран размер обучающей выборки, который равен количеству строк в датафрейме X_train
    print('train size: {}'.format(X_train.shape[0]))
    # Вычисляем и выводим на экран долю мошеннических транзакций в обучающей выборке, которая называется загрязнением (contamination)
    # Для этого используем метод value_counts с параметром normalize=True, который возвращает частоту каждого уникального значения в серии y_train
    # Затем выбираем значение с индексом 1, которое соответствует мошенническим транзакциям, и округляем его до трех знаков после запятой
    contamination = round(y_train.value_counts(normalize=True)[1], 3)
    # Выводим на экран значение загрязнения в процентах
    print('contamination: {} ({}%)'.format(contamination, contamination * 100))
    # Возвращаем из функции три объекта: X_train, y_train и contamination
    return X_train, y_train, contamination



In [ ]:

  # Вызываем функцию get_data_part с датафреймом X_norm, который содержит стандартизованные признаки транзакций, серией y,
# которая содержит метки классов, и параметром size, который задает долю данных, которую мы хотим взять
# Параметр size можно изменить в зависимости от того, сколько данных мы хотиm использовать для обучения моделей
# Сохраняем результаты функции в трех переменных: X_train, y_train и contamination
X_train, y_train, contamination = get_data_part(X_norm, y, size=0.1)



In [ ]:
contamination # доля мошеннических транзакций в обучающей выборке равна 0.002

In [ ]:
# Метки классов для тестовой выборки
# y_train это переменная, которая содержит зависимые переменные, называемые целевыми, для обучения модели машинного обучения.
# Это то, что вы пытаетесь предсказать или объяснить с помощью вашей модели
y_train.head()

> y_train - это серия, которая содержит метки классов для транзакций, которые были выбраны для обучающей выборки. Метки классов показывают, является ли транзакция мошеннической или нет. В серии y_train есть два возможных значения: 0 - не мошенническая, 1 - мошенническая. Серия y_train имеет тот же размер и индексы, что и датафрейм X_train, который содержит признаки транзакций для обучающей выборки. Серия y_train была получена с помощью функции get_data_part, которую мы определили ранее.

In [ ]:
type(y_train) # Объект Серия

In [ ]:
# Просмотр
y_train.describe()

> Можно сделать следующие выводы:

Количество наблюдений в Series равно 28480.
Среднее значение переменной Class равно 0.001791, что означает, что доля мошеннических транзакций в Series очень мала.
Стандартное отклонение переменной Class равно 0.042280, что означает, что значения переменной Class сильно разбросаны вокруг среднего.
Минимальное и максимальное значение переменной Class равны 0 и 1 соответственно, что подтверждает, что это бинарный признак.
25%, 50% и 75% квантили переменной Class равны 0, что означает, что большинство транзакций в Series являются нормальными. Только 25% транзакций имеют значение переменной Class равное 1, что означает, что они являются мошенническими.

In [ ]:
# Выводим на экран первые пять строк датафрейма X_train, который содержит признаки транзакций для обучающей выборки
X_train.head()


## Модели PyOD

In [ ]:
%pip install pyod
# Я пока закоментирую.

In [ ]:
# Импортируем функцию evaluate_print из модуля pyod.utils.data, который содержит полезные утилиты для работы с данными и моделями детекции выбросов
from pyod.utils.data import evaluate_print

In [ ]:
%pip install optuna
# Я пока закоментирую.

In [ ]:
# Импортируем библиотеку Optuna, которая предназначена для автоматического подбора оптимальных гиперпараметров
# для различных моделей машинного обучения
import optuna

# Импортируем функции plot_param_importances и plot_parallel_coordinate из модуля optuna.visualization.matplotlib,
# которые позволяют визуализировать важность параметров и параллельные координаты с помощью библиотеки matplotlib
from optuna.visualization.matplotlib import plot_param_importances, plot_parallel_coordinate

# Импортируем функцию average_precision_score из модуля sklearn.metrics, которая вычисляет среднюю точность
# (average precision, AP) из прогнозных оценок
# AP является одной из метрик для оценки качества детекции выбросов (лайфхак)
from sklearn.metrics import average_precision_score

# Импортируем функцию confusion_matrix из модуля sklearn.metrics, которая вычисляет матрицу ошибок (confusion matrix)
# из истинных и предсказанных меток классов
# Матрица ошибок является другой метрикой для оценки качества детекции выбросов
from sklearn.metrics import confusion_matrix

# Импортируем класс ConfusionMatrixDisplay из модуля sklearn.metrics, который позволяет визуализировать матрицу
# ошибок с помощью библиотеки matplotlib
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
# Определяем функцию optimize, которая принимает на вход функцию objective, которая задает целевую функцию для оптимизации,
# и параметр n, который задает количество испытаний для оптимизации
def optimize(objective, n):
    # Импортируем библиотеку Optuna для автоматического подбора гиперпараметров
    import optuna
    # Создаем объект study, который будет представлять процесс оптимизации
    # Параметр direction='maximize' означает, что мы хотим максимизировать значение целевой функции
    study = optuna.create_study(direction='maximize')

    # Запускаем оптимизацию с помощью метода optimize, который принимает на вход функцию objective, параметр n_trials,
    # который задает количество испытаний для оптимизации, и параметр show_progress_bar, который показывает индикатор прогресса
    study.optimize(objective, n_trials=n, show_progress_bar=True)

    # Выводим на экран лучшие параметры и лучшее значение целевой функции, которые были найдены в процессе оптимизации
    print('\nBest params:{}\nBest value:{}'.format(study.best_params, study.best_value))

    # Импортируем функцию plot_param_importances из модуля optuna.visualization.matplotlib, которая позволяет визуализировать
    # важность параметров с помощью библиотеки matplotlib
    from optuna.visualization.matplotlib import plot_param_importances

    # Строим график важности параметров с помощью функции plot_param_importances, которая принимает на вход объект study
    plot_param_importances(study)

    # Импортируем функцию plot_parallel_coordinate из модуля optuna.visualization.matplotlib, которая позволяет визуализировать
    # параллельные координаты с помощью библиотеки matplotlib

    from optuna.visualization.matplotlib import plot_parallel_coordinate
    # Строим график параллельных координат с помощью функции plot_parallel_coordinate, которая принимает на вход объект study
    plot_parallel_coordinate(study)

In [ ]:
# Определяем функцию metrics, которая принимает на вход модель кластеризации и истинные метки классов
def metrics(model, y_train):
    # Устанавливаем случайное зерно для воспроизводимости результатов
    np.random.seed(42)
    # Предсказываем метки классов с помощью модели
    y_pred = model.labels_
    # Выводим на экран название модели и метрики качества кластеризации: гомогенность, полноту и V-меру
    evaluate_print(model.__class__, y_train, y_pred)
    # Выводим на экран отчет о классификации, который содержит точность, полноту, F-меру и поддержку для каждого класса
    print(classification_report(y_train, y_pred))
    # Вычисляем матрицу ошибок, которая показывает, сколько объектов каждого класса было правильно или неправильно отнесено к другому классу
    cm = confusion_matrix(y_train, y_pred)
    # Выводим на экран матрицу ошибок
    print(cm)
    # Создаем объект для визуализации матрицы ошибок с помощью библиотеки matplotlib
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    # Строим график матрицы ошибок
    disp.plot()

In [ ]:
# Устанавливаем случайное зерно для воспроизводимости результатов
np.random.seed(42)

### kNN Example

 > kNN - это алгоритм машинного обучения, который используется для решения задач классификации и регрессии. Он основан на идее, что объекты с похожими признаками имеют похожие метки классов или значения. kNN работает так: для нового объекта он находит k ближайших соседей в обучающей выборке, то есть те объекты, которые имеют наименьшее расстояние до нового объекта. Затем он присваивает новому объекту метку класса или значение, которое является наиболее частым среди его соседей.

In [ ]:
from pyod.models.knn import KNN

# Импортируем библиотеку pyod, которая предоставляет различные алгоритмы для обнаружения аномалий в данных
from pyod.models.knn import KNN
# Импортируем класс KNN из модуля pyod.models.knn, который реализует алгоритм kNN для обнаружения аномалий
# Алгоритм kNN для обнаружения аномалий работает так:
# для каждого объекта он находит k ближайших соседей в обучающей выборке, то есть те объекты, которые имеют
# наименьшее расстояние до данного объекта. Затем он вычисляет аномальный коэффициент для каждого объекта как
# среднее расстояние до его соседей. Чем больше аномальный коэффициент, тем больше вероятность, что объект является аномалией.

In [ ]:
# Мы вызываем функцию get_data_part, которая принимает три аргумента:
# X_norm, y и size.
# X_norm - это массив, который содержит нормализованные значения признаков из исходных данных,
# y - это массив, который содержит метки классов для каждого объекта из исходных данных,
# а size - это размер выборки, которую мы хотим получить из исходных данных.
# Функция get_data_part возвращает три значения:
# X_train, y_train и contamination.
# X_train - это массив, который содержит объекты из X_norm, выбранные случайным образом,
# y_train - это массив, который содержит метки классов из y, соответствующие выбранным объектам,
# а contamination - это доля объектов класса 1 в выборке.
X_train, y_train, contamination = get_data_part(X_norm, y, size=0.2);

> Вывод train size: 56961
contamination: 0.002 (0.2%)  Это значит, что мы обучаем модель на выборке из 56961 объектов, из которых 0.002 (0.2%) принадлежат классу 1, а остальные - классу 0. Класс 1 - это аномальные или необычные объекты, а класс 0 - это нормальные или типичные объекты. Мы хотим, чтобы модель могла обнаруживать аномалии в новых данных, которые не входят в выборку.

Теоретическая справка
 Аномалии - это объекты, которые отличаются от большинства других объектов по своим признакам или поведению. Например, аномалией может быть мошенническая транзакция, неисправный датчик, редкий вид животного или необычный текст. Мы хотим, чтобы модель могла определить, является ли новый объект аномалией или нет, исходя из его сходства с обучающей выборкой.

In [ ]:
# Определяем функцию objective, которая принимает на вход объект trial, который представляет одно испытание оптимизации
def objective(trial):
    # Устанавливаем случайное зерно для воспроизводимости результатов
    np.random.seed(42)
    # Создаем объект модели kNN для обнаружения аномалий с помощью библиотеки pyod
    clf = KNN(
        # Задаем долю аномалий в обучающей выборке
        contamination=contamination,
        # Задаем количество соседей для kNN с помощью метода suggest_int, который выбирает случайное целое число
        # из логарифмического распределения от 50 до 1000
        n_neighbors=trial.suggest_int('n', 50, 1000, log=True),
        # Задаем метод агрегации расстояний до соседей с помощью метода suggest_categorical,
        # который выбирает случайное значение из списка ['largest', 'mean', 'median']
        method=trial.suggest_categorical('method', ['largest', 'mean', 'median']),
        # Задаем радиус для kNN с помощью метода suggest_float, который выбирает случайное вещественное число
        # из равномерного распределения от 0.5 до 1
        radius=trial.suggest_float('radius', 0.5, 1),
        # Задаем метрику расстояния для kNN с помощью метода suggest_categorical, который выбирает случайное значение
        # из списка ['l1', 'minkowski', 'l2']
        metric=trial.suggest_categorical('metric', ['l1', 'minkowski', 'l2'])
    )
    # Обучаем модель на обучающей выборке X_train
    clf.fit(X_train)
    # Предсказываем метки классов для обучающей выборки X_train
    y_pred = clf.labels_
    # Возвращаем значение метрики качества обнаружения аномалий - среднюю точность (average precision score),
    # которая вычисляется как площадь под кривой точность-полнота (precision-recall curve)
    return average_precision_score(y_train, y_pred)

> Пояснение:
metric=trial.suggest_categorical('metric', ['l1', 'minkowski', 'l2'])

- `metric` - это имя параметра, который мы хотим оптимизировать. Он задает метрику расстояния для kNN, то есть способ измерения, насколько близки или далеки объекты друг от друга в пространстве признаков.
- `trial` - это объект, который представляет одно испытание оптимизации. Он содержит методы, которые позволяют выбирать случайные значения для параметров из разных распределений или списков.
- `suggest_categorical` - это метод, который принимает на вход имя параметра и список возможных значений для него. Он выбирает случайное значение из списка и возвращает его. Это значение будет использоваться для обучения модели в данном испытании.
- `('metric', ['l1', 'minkowski', 'l2'])` - это аргументы, которые передаются в метод `suggest_categorical`. Они задают имя параметра `metric` и список возможных значений для него: `['l1', 'minkowski', 'l2']`. Это разные метрики расстояния, которые могут быть использованы для kNN. `l1` - это манхэттенское расстояние, `minkowski` - это обобщенное расстояние, которое включает в себя евклидово и манхэттенское расстояния в зависимости от параметра `p`, `l2` - это евклидово расстояние.
Мы выбираем случайное значение из списка `['l1', 'minkowski', 'l2']` и присваивает его параметру `metric`. Это значение будет использоваться для вычисления расстояний между объектами в kNN.

In [ ]:
# Вызываем функцию optimize, которая принимает на вход функцию objective, которая задает целевую функцию для оптимизации,
# и параметр n, который задает количество испытаний для оптимизации
# 20 - это количество испытаний, которые мы хотим провести для оптимизации параметров модели обнаружения аномалий.
# Каждое испытание выбирает случайные значения для параметров модели из заданных диапазонов или списков с помощью
# методов suggest_int, suggest_categorical и suggest_float. Затем оно обучает модель на обучающей выборке и вычисляет
# значение целевой функции, которая равна средней точности (average precision score) на обучающей выборке.
# Средняя точность - это метрика качества обнаружения аномалий, которая вычисляется как площадь под кривой точность-полнота
# (precision-recall curve). Чем выше средняя точность, тем лучше модель обнаруживает аномалии.
# После проведения всех испытаний мы можем увидеть лучшие параметры и лучшее значение целевой функции,
# которые были найдены в процессе оптимизации, а также два графика, которые показывают важность параметров
# и параллельные координаты. Важность параметров показывает, насколько каждый параметр влияет на значение целевой функции,
# а параллельные координаты показывают, как параметры и значение целевой функции меняются в разных испытаниях.
# Эти графики помогут нам понять, какие параметры лучше всего подходят для нашей модели.
optimize(objective, 20)

**Вывод**
> Выводим на экран лучшие параметры и лучшее значение целевой функции, которые были найдены в процессе оптимизации с помощью функции optimize.
Параметры меняются с обновлением:
Best params:{'n': , 'method': 'median', 'radius': , 'metric': 'l1'}
Best value:0.07
Best params - это словарь с лучшими параметрами для модели kNN, которые были выбраны в ходе оптимизации. Они задают количество соседей (n), метод агрегации расстояний до соседей (method), радиус для kNN (radius) и метрику расстояния (metric) l2 - это евклидово расстояние.
Best value - это лучшее значение целевой функции, которая вычисляет среднюю точность (average precision score) для модели kNN на обучающей выборке. Средняя точность - это метрика качества обнаружения аномалий, которая вычисляется как площадь под кривой точность-полнота (precision-recall curve). Чем выше средняя точность, тем лучше модель обнаруживает аномалии.
0.07 - это числовое значение средней точности, которое было получено для модели kNN с лучшими параметрами. Оно показывает, что модель достаточно хорошо обнаруживает аномалии, но есть еще простор для улучшения.

### LOF
Это реализация метода обнаружения аномалий, который называется локальный коэффициент выброса (LOF). Этот метод вычисляет отклонение локальной плотности данной точки данных относительно ее соседей. Он считает выбросами те объекты, которые имеют значительно меньшую плотность, чем их соседи.

PyOD LOF - это обертка класса scikit-learn LOF. PyOD - это библиотека Python для обнаружения аномалий, которая предоставляет различные алгоритмы и инструменты для этой задачи. Scikit-learn - это библиотека Python для машинного обучения, которая содержит множество моделей и методов для анализа данных. Класс LOF - это реализация метода LOF в scikit-learn, которая позволяет обучать модель и предсказывать аномалии.

In [ ]:
# Импортируем библиотеку pyod, которая предоставляет различные алгоритмы для обнаружения аномалий в данных
from pyod.models.lof import LOF
# Импортируем класс LOF из модуля pyod.models.lof, который реализует метод локального коэффициента выброса (LOF)
# для обнаружения аномалий
# Метод LOF вычисляет отклонение локальной плотности данной точки данных относительно ее соседей.
# Он считает выбросами те объекты, которые имеют значительно меньшую плотность, чем их соседи.

In [ ]:
# Мы вызываем функцию get_data_part, которая принимает три аргумента:
# X_norm, y и size.
# X_norm - это массив, который содержит нормализованные значения признаков из исходных данных,
# y - это массив, который содержит метки классов для каждого объекта из исходных данных,
# а size - это размер выборки, которую мы хотим получить из исходных данных.
# Функция get_data_part возвращает три значения:
# X_train, y_train и contamination.
# X_train - это массив, который содержит объекты из X_norm, выбранные случайным образом,
# y_train - это массив, который содержит метки классов из y, соответствующие выбранным объектам,
# а contamination - это доля объектов класса 1 в выборке.
X_train, y_train, contamination = get_data_part(X_norm, y, size=0.2)

In [ ]:
# Определяем функцию objective, которая принимает на вход объект trial, который представляет одно испытание оптимизации
def objective(trial):
    # Устанавливаем случайное зерно для воспроизводимости результатов
    np.random.seed(42)
    # Создаем объект модели LOF для обнаружения аномалий с помощью библиотеки pyod
    clf = LOF(
        # Задаем долю аномалий в обучающей выборке
        contamination=contamination,
        # Задаем количество соседей для LOF с помощью метода suggest_int, который выбирает случайное целое число
        # из логарифмического распределения от 50 до 1000
        n_neighbors=trial.suggest_int('n_neighbors', 50, 1000, log=True),
        # Задаем метрику расстояния для LOF с помощью метода suggest_categorical, который выбирает случайное значение
        # из списка ['l2', 'l1', 'minkowski']
        metric=trial.suggest_categorical('metric', ['l2', 'l1', 'minkowski'])
    )
    # Обучаем модель на обучающей выборке X_train
    clf.fit(X_train)
    # Предсказываем метки классов для обучающей выборки X_train
    y_pred = clf.labels_
    # Возвращаем значение метрики качества обнаружения аномалий - среднюю точность (average precision score),
    # которая вычисляется как площадь под кривой точность-полнота (precision-recall curve)
    return average_precision_score(y_train, y_pred)

In [ ]:
# Вызываем функцию optimize, которая принимает на вход функцию objective, которая задает целевую функцию для оптимизации,
# и параметр n, который задает количество испытаний для оптимизации
# 12 - это количество испытаний, которые мы хотим провести для оптимизации параметров модели обнаружения аномалий.
# Каждое испытание выбирает случайные значения для параметров модели из заданных диапазонов или списков с помощью
# методов suggest_int, suggest_categorical и suggest_float. Затем оно обучает модель на обучающей выборке и вычисляет
# значение целевой функции, которая равна средней точности (average precision score) на обучающей выборке.
# Средняя точность - это метрика качества обнаружения аномалий, которая вычисляется как площадь под кривой точность-полнота
# (precision-recall curve). Чем выше средняя точность, тем лучше модель обнаруживает аномалии.
# После проведения всех испытаний мы можем увидеть лучшие параметры и лучшее значение целевой функции,
# которые были найдены в процессе оптимизации, а также два графика, которые показывают важность параметров
# и параллельные координаты. Важность параметров показывает, насколько каждый параметр влияет на значение целевой функции,
# а параллельные координаты показывают, как параметры и значение целевой функции меняются в разных испытаниях.
# Эти графики помогут нам понять, какие параметры лучше всего подходят для нашей модели.
optimize(objective, 12)

**Вывод**
Лучшие параметры для модели LOF, которые были найдены в процессе оптимизации, это:
Параметры меняются с обновлением:
- `n_neighbors`: - это количество соседей, которые используются для вычисления локальной плотности каждого объекта.
- `metric`: 'minkowski' - это метрика расстояния, которая используется для измерения близости или дальности объектов в пространстве признаков. Метрика 'minkowski' - это обобщенная метрика, которая включает в себя евклидово и манхэттенское расстояния в зависимости от параметра `p`.
-  лучшее значение целевой функции, которая вычисляет среднюю точность (average precision score) для модели LOF на обучающей выборке, равно 0.2.
- Средняя точность - это метрика качества обнаружения аномалий, которая вычисляется как площадь под кривой точность-полнота (precision-recall curve). Чем выше средняя точность, тем лучше модель обнаруживает аномалии.
- Значение 0.2 показывает, что модель достаточно хорошо обнаруживает аномалии.

In [ ]:
# Пояснения: этот код выполняться будет долго, пока закоментируем его
#%%time
 ###### Используем магическую команду %%time, которая позволяет измерить время выполнения ячейки кода в Jupyter Notebook

 ###### Создаем объект модели LOF для обнаружения аномалий с помощью библиотеки pyod
#clf = LOF(
# Задаем долю аномалий в обучающей выборке
   # contamination=contamination,
# Задаем количество соседей для LOF, равное 280
    # neighbors=280,###### Задаем метрику расстояния для LOF, равную l1, то есть манхэттенскому расстоянию
    # metric='l1'
# )
 ###### Обучаем модель на обучающей выборке X_train
#clf.fit(X_train)

In [ ]:
# metrics(clf, y_train)

 > Вызоваем функцию metrics, которая оценивает качество модели обнаружения аномалий на обучающей выборке.
 - Функция metrics принимает на вход модель clf, которая была создана и обучена с помощью библиотеки pyod, и метки классов y_train, которые показывают, какие объекты являются аномалиями (класс 1) или нормальными (класс 0).
- Функция metrics вычисляет и выводит на экран разные метрики, которые показывают, насколько хорошо модель clf согласуется с истинными метками классов. Эти метрики включают гомогенность, полноту, V-меру, точность, полноту, F-меру и поддержку для каждого класса. Функция metrics также строит и выводит на экран матрицу ошибок, которая показывает, сколько объектов каждого класса было правильно или неправильно отнесено к другому классу.

> Теоретическая справка
Кластеризация - это задача разбиения данных на группы (кластеры) так, чтобы объекты внутри одного кластера были похожи друг на друга, а объекты из разных кластеров - различны. Метки классов - это истинные принадлежности объектов к кластерам, которые могут быть известны или неизвестны. Модель кластеризации - это алгоритм, который пытается найти оптимальное разбиение данных на кластеры. Модель может предсказать метки кластеров для новых объектов, которые не входят в обучающую выборку.

 > Метрики:
 - Гомогенность - это мера того, насколько каждый кластер содержит только объекты одного класса. Гомогенность равна 1, если все кластеры гомогенны, и меньше 1, если в кластерах есть объекты разных классов.
- Полнота - это мера того, насколько все объекты одного класса попадают в один кластер. Полнота равна 1, если все объекты одного класса собраны в один кластер, и меньше 1, если объекты одного класса разбросаны по разным кластерам.
- V-мера - это среднее гармоническое между гомогенностью и полнотой. V-мера равна 1, если и гомогенность, и полнота равны 1, и меньше 1, если одна из них меньше 1. V-мера показывает, насколько хорошо модель кластеризации соответствует истинному разбиению данных на классы.
- Точность - это доля правильно предсказанных меток классов среди всех предсказанных меток. Точность равна 1, если все метки предсказаны верно, и меньше 1, если есть ошибки в предсказаниях. Точность показывает, насколько точно модель кластеризации определяет принадлежность объектов к классам.
- Полнота - это доля правильно предсказанных меток классов среди всех истинных меток. Полнота равна 1, если все истинные метки предсказаны верно, и меньше 1, если есть пропуски в предсказаниях. Полнота показывает, насколько полно модель кластеризации учитывает все объекты каждого класса.
- F-мера - это среднее гармоническое между точностью и полнотой. F-мера равна 1, если и точность, и полнота равны 1, и меньше 1, если одна из них меньше 1. F-мера показывает, насколько хорошо модель кластеризации сбалансирована между точностью и полнотой.
- Поддержка - это количество объектов каждого класса в обучающей выборке. Поддержка показывает, насколько представительны данные для каждого класса.

### Isolation Forest

- Isolation Forest - это алгоритм для обнаружения аномалий в данных, который использует бинарные деревья для изоляции аномальных точек на основе их длины пути. Он имеет линейную временную сложность, низкое требование к памяти и хорошо работает с высокоразмерными данными.
- Isolation Forest работает так: для каждого объекта он находит k ближайших соседей в обучающей выборке, то есть те объекты, которые имеют наименьшее расстояние до данного объекта. Затем он вычисляет аномальный коэффициент для каждого объекта как среднее расстояние до его соседей. Чем больше аномальный коэффициент, тем больше вероятность, что объект является аномалией.

In [ ]:
pip install pyod

In [ ]:
# импортируем модуль pyod.models.iforest, который содержит класс IForest для обнаружения аномалий с помощью изолирующего леса
# изолирующий лес - это алгоритм машинного обучения, который строит множество случайных деревьев и измеряет, насколько легко изолировать каждую точку данных
# чем больше аномальна точка данных, тем меньше разделений ей нужно, чтобы быть изолированной от остальных
from pyod.models.iforest import IForest

In [ ]:
# задаем нормализованные данные X_norm и метки y, которые хотим разделить

# вызываем функцию get_data_part с аргументами X_norm, y и size
# X_norm - это двумерный массив с данными
# y - это одномерный массив с метками, где 0 - нормальное значение, а 1 - аномальное
# size - это доля данных, которая будет использована для обучающей выборки
# функция возвращает три значения: X_train, y_train и contamination
# X_train - это двумерный массив с обучающими данными
# y_train - это одномерный массив с обучающими метками
# contamination - это доля аномалий в обучающей выборке

X_train, y_train, contamination = get_data_part(X_norm, y, size=0.3)

- Размер выборки 85442
- доля аномаоий 0.002

In [ ]:
# определяем функцию objective, которая принимает аргумент trial
def objective(trial):
    # создаем объект класса IForest, который использует алгоритм изолирующего леса для обнаружения аномалий
    # contamination - это доля аномалий в данных, которая задается заранее
    # random_state - это параметр для воспроизводимости результатов
    # n_estimators - это количество деревьев в изолирующем лесе, которое подбирается с помощью метода suggest_int
    # метод suggest_int принимает имя параметра, нижнюю и верхнюю границы и шаг для выбора целочисленного значения
    # параметр log=True означает, что значения выбираются из логарифмического пространства
    clf = IForest(
        contamination=contamination,
        random_state=0,
        n_estimators=trial.suggest_int('n', 50, 1000, log=True)
    )
    # обучаем модель на обучающих данных X_train
    clf.fit(X_train)
    # получаем предсказанные метки для обучающих данных
    y_pred = clf.labels_
    # возвращаем значение метрики average_precision_score, которая вычисляет среднюю точность по кривой точность-полнота
    # метрика принимает истинные метки y_train и предсказанные метки y_pred
    return average_precision_score(y_train, y_pred)

In [ ]:
# Вызываем функцию optimize, которая принимает на вход функцию objective, которая задает целевую функцию для оптимизации,
# и параметр n, который задает количество испытаний для оптимизации
# 10 - это количество испытаний, которые мы хотим провести для оптимизации параметров модели обнаружения аномалий.
# Каждое испытание выбирает случайные значения для параметров модели из заданных диапазонов или списков с помощью
# методов suggest_int, suggest_categorical и suggest_float. Затем оно обучает модель на обучающей выборке и вычисляет
# значение целевой функции, которая равна средней точности (average precision score) на обучающей выборке.
# Средняя точность - это метрика качества обнаружения аномалий, которая вычисляется как площадь под кривой точность-полнота
# (precision-recall curve). Чем выше средняя точность, тем лучше модель обнаруживает аномалии.
# После проведения всех испытаний мы можем увидеть лучшие параметры и лучшее значение целевой функции,
# которые были найдены в процессе оптимизации, а также два графика, которые показывают важность параметров
# и параллельные координаты. Важность параметров показывает, насколько каждый параметр влияет на значение целевой функции,
# а параллельные координаты показывают, как параметры и значение целевой функции меняются в разных испытаниях.
# Эти графики помогут нам понять, какие параметры лучше всего подходят для нашей модели.
optimize(objective, 10)

**Вывод**
- Наша модель достигла наивысшего значения метрики average_precision_score, которая измеряет качество обнаружения аномалий.
Параметры меняются с обновлением:
Лучшие параметры - это {'n': }, что означает, что изолирующий лес состоит из n деревьев.
- Лучшее значение - это 0.09, что означает, что средняя точность по кривой точность-полнота составляет примерно n %. Это довольно низкое значение, что может свидетельствовать о том, что  данные сложны для обнаружения аномалий или что нужно попробовать другой метод или другие параметры.

### HBOS

- HBOS - это сокращение от Histogram-based Outlier Score, что означает оценка аномалий на основе гистограмм.
- Это эффективный метод обнаружения аномалий без учителя. Он предполагает, что признаки независимы и вычисляет степень выброса, построив гистограммы для каждого признака. Этот методреализован в библиотеке PyOD.

In [ ]:
# импортируем модуль pyod.models.hbos, который содержит класс HBOS для обнаружения аномалий с помощью гистограмм
# HBOS - это сокращение от Histogram-based Outlier Score, что означает оценка аномалий на основе гистограмм
# Это эффективный метод обнаружения аномалий без учителя, который работает быстрее, чем многомерные методы, но менее точно

from pyod.models.hbos import HBOS

In [ ]:
# задаем нормализованные данные X_norm и метки y, которые хотим разделить

# вызываем функцию get_data_part с аргументами X_norm, y и size
# X_norm - это двумерный массив с данными
# y - это одномерный массив с метками, где 0 - нормальное значение, а 1 - аномальное
# size - это доля данных, которая будет использована для обучающей выборки
# функция возвращает три значения: X_train, y_train и contamination
# X_train - это двумерный массив с обучающими данными
# y_train - это одномерный массив с обучающими метками
# contamination - это доля аномалий в обучающей выборке
X_train, y_train, contamination = get_data_part(X_norm, y, size=0.3)

- Размер выборки 85442
- доля аномаоий 0.002

In [ ]:
# определяем функцию objective, которая принимает аргумент trial
def objective(trial):
    # создаем объект класса HBOS, который использует алгоритм обнаружения аномалий на основе гистограмм
    # contamination - это доля аномалий в данных, которая задается заранее
    # n_bins - это количество бинов для построения гистограмм, 'auto' означает, что оно будет определено автоматически
    # alpha - это параметр, который контролирует ширину бинов, чем больше alpha, тем меньше бинов
    # tol - это параметр, который контролирует порог для определения аномалий, чем больше tol, тем больше аномалий
    # alpha и tol подбираются с помощью метода suggest_float, который возвращает случайное вещественное число
    # из заданного интервала
    # параметр log=True означает, что значения выбираются из логарифмического пространства
    clf = HBOS(
        contamination=contamination,
        n_bins='auto',
        alpha=trial.suggest_float('alpha', 0.05, 0.2),
        tol=trial.suggest_float('tol', 0.05, 0.9, log=True)
    )
    # обучаем модель на обучающих данных X_train
    clf.fit(X_train)
    # получаем предсказанные метки для обучающих данных
    y_pred = clf.labels_
    # возвращаем значение метрики average_precision_score, которая вычисляет среднюю точность по кривой точность-полнота
    # метрика принимает истинные метки y_train и предсказанные метки y_pred

    return average_precision_score(y_train, y_pred)

In [ ]:
# Вызываем функцию optimize, которая принимает на вход функцию objective, которая задает целевую функцию для оптимизации,
# и параметр n, который задает количество испытаний для оптимизации
# 12 - это количество испытаний, которые мы хотим провести для оптимизации параметров модели обнаружения аномалий.
# Каждое испытание выбирает случайные значения для параметров модели из заданных диапазонов или списков с помощью
# методов suggest_int, suggest_categorical и suggest_float. Затем оно обучает модель на обучающей выборке и вычисляет
# значение целевой функции, которая равна средней точности (average precision score) на обучающей выборке.
# Средняя точность - это метрика качества обнаружения аномалий, которая вычисляется как площадь под кривой точность-полнота
# (precision-recall curve). Чем выше средняя точность, тем лучше модель обнаруживает аномалии.
# После проведения всех испытаний мы можем увидеть лучшие параметры и лучшее значение целевой функции,
# которые были найдены в процессе оптимизации, а также два графика, которые показывают важность параметров
# и параллельные координаты. Важность параметров показывает, насколько каждый параметр влияет на значение целевой функции,
# а параллельные координаты показывают, как параметры и значение целевой функции меняются в разных испытаниях.
# Эти графики помогут нам понять, какие параметры лучше всего подходят для нашей модели.

optimize(objective, 12)

**Вывод **
Параметры меняются с обновлением:
- Лучшие параметры для модели HBOS.
наша модель достигла наивысшего значения метрики average_precision_score, которая измеряет качество обнаружения аномалий. - - Лучшие параметры - это {'alpha': 0.051, 'tol': 0.50}, что означает, что гистограмма имеет ширину бинов 0.0511 и порог для определения аномалий 0.50.
- Лучшее значение - это 0.107, что означает, что средняя точность по кривой точность-полнота составляет примерно 10.7%.
Это немного лучше, чем с моделью IForest, но все еще довольно низкое значение, что может свидетельствовать о том, что данные сложны для обнаружения аномалий или что нужно попробовать другой метод или другие параметры.

### Parameter-free models

Parameter-free models - это модели, которые не требуют никаких параметров, заданных пользователем, для выполнения обнаружения аномалий. Они также называются непараметрическими моделями, потому что они не предполагают никакого конкретного распределения для данных. Parameter-free models полезны, когда данные сложны, высокоразмерны или имеют неизвестные характеристики.
- Unsupervised Outlier Detection Using Empirical Cumulative Distribution Functions (ECOD): Эта модель использует эмпирическую кумулятивную функцию распределения (ECDF) каждого признака для измерения выброса каждой точки данных. ECDF - это доля точек данных, которые меньше или равны заданному значению. Модель вычисляет ECDF-оценку для каждой точки данных как произведение ECDF-значений всех признаков, а затем ранжирует точки данных по их ECDF-оценкам. Чем ниже ECDF-оценка, тем больше вероятность, что точка данных является аномальной.
- Rotation-based Outlier Detector (ROD): Эта модель использует метод, основанный на повороте, для преобразования данных в пространство меньшей размерности, где выбросы более заметны. Модель поворачивает данные так, чтобы максимизировать разницу между средними значениями нормальных и аномальных точек в каждом направлении. Затем модель вычисляет ROD-оценку для каждой точки данных как сумму квадратов расстояний от средних значений в каждом направлении. Чем выше ROD-оценка, тем больше вероятность, что точка данных является аномальной.
- Copula Based Outlier Detector (COPOD): Эта модель использует копулы для моделирования зависимости между признаками данных. Копула - это функция, которая связывает маргинальные распределения признаков в совместное распределение. Модель вычисляет COPOD-оценку для каждой точки данных как произведение верхней и нижней копулы, которые измеряют вероятность того, что точка данных превышает или меньше остальных точек по всем признакам. Чем выше COPOD-оценка, тем больше вероятность, что точка данных является аномальной.

In [ ]:
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.rod import ROD

In [ ]:
# импортируем модуль pyod.models.ecod, который содержит класс ECOD для обнаружения аномалий с помощью эмпирических кумулятивных
# функций распределения
# ECOD - это сокращение от Empirical-Cumulative-distribution-based Outlier Detection, что означает оценка аномалий на
# основе эмпирических кумулятивных функций распределения
# Это эффективный метод обнаружения аномалий без учителя, который работает быстрее, чем многомерные методы, но менее точно.

from pyod.models.ecod import ECOD

# импортируем модуль pyod.models.copod, который содержит класс COPOD для обнаружения аномалий с помощью копул
# COPOD - это сокращение от Copula-Based Outlier Detection, что означает оценка аномалий на основе копул
# Это новый метод обнаружения аномалий без учителя, который вдохновлен статистическими методами для моделирования
# многомерного распределения данных

from pyod.models.copod import COPOD

# импортируем модуль pyod.models.rod, который содержит класс ROD для обнаружения аномалий с помощью метода, основанного
# на повороте
# ROD - это сокращение от Rotation-based Outlier Detector, что означает детектор аномалий, основанный на повороте
# Это еще один новый метод обнаружения аномалий без учителя, который использует технику поворота для преобразования
# данных в пространство меньшей размерности, где выбросы более заметны

from pyod.models.rod import ROD

In [ ]:
# задаем нормализованные данные X_norm и метки y, которые хотим разделить

# вызываем функцию get_data_part с аргументами X_norm, y и size
# X_norm - это двумерный массив с данными
# y - это одномерный массив с метками, где 0 - нормальное значение, а 1 - аномальное
# size - это доля данных, которая будет использована для обучающей выборки
# функция возвращает три значения: X_train, y_train и contamination
# X_train - это двумерный массив с обучающими данными
# y_train - это одномерный массив с обучающими метками
# contamination - это доля аномалий в обучающей выборке
X_train, y_train, contamination = get_data_part(X_norm, y, size=0.8)

- Размер выборки 227845
- доля аномаоий 0.002

In [ ]:
%%time
# Используем магическую команду %%time для измерения времени выполнения ячейки

# Создаем объект класса ECOD, который использует алгоритм обнаружения аномалий на основе эмпирических кумулятивных функций
# распределения
# contamination - это доля аномалий в данных, которая задается заранее
ecod = ECOD(contamination=contamination)
# Обучаем модель на обучающих данных X_train
ecod.fit(X_train)
# Вызываем функцию metrics, которая вычисляет и выводит различные метрики качества модели, такие как точность,
# полнота, F1-мера и AUC
# Функция принимает два аргумента: модель и истинные метки y_train
metrics(ecod, y_train)

**Вывод** , ниже описано как модель ECOD справилась с обнаружением аномалий на обучающих данных.
- ROC: это сокращение от Receiver Operating Characteristic, что означает характеристика работы приемника. Это кривая, которая показывает, как меняется соотношение между долей верно классифицированных аномалий (True Positive Rate) и долей неверно классифицированных нормальных точек (False Positive Rate) при изменении порога для определения аномалий. Чем выше ROC, тем лучше модель разделяет аномалии и нормальные точки. К сожалению, наша модель имеет ROC 0.6668, что означает, что она не очень хорошо справляется с этой задачей.
- Precision: это доля точек данных, которые модель правильно определила как аномальные, среди всех точек, которые модель определила как аномальные. Чем выше precision, тем меньше ложных срабатываний модели. Модель имеет precision 0.29, что означает, что из всех точек, которые модель пометила как аномальные, только 29% действительно являются аномальными.
- Recall: это доля точек данных, которые модель правильно определила как аномальные, среди всех аномальных точек в данных. Чем выше recall, тем меньше пропущенных аномалий моделью. Модель имеет recall 0.34, что означает, что из всех аномальных точек в данных, модель нашла только 34%.
- F1-score: это среднее гармоническое между precision и recall, которое учитывает их баланс. Чем выше F1-score, тем лучше модель согласовывает precision и recall. Модель имеет F1-score 0.31, что означает, что она не очень хорошо балансирует между precision и recall.
- Confusion matrix: это таблица, которая показывает, сколько точек данных было правильно или неправильно классифицировано моделью. Модель имеет confusion matrix [[227129    325]
 [   260    131]], что означает, что она правильно определила 227129 нормальных точек и 131 аномальную точку, но также ошибочно определила 325 нормальных точек как аномальные и пропустила 260 аномальных точек.

In [ ]:
%%time
# Используем магическую команду %%time для измерения времени выполнения ячейки

# Создаем объект класса COPOD, который использует алгоритм обнаружения аномалий на основе копул
# contamination - это доля аномалий в данных, которая задается заранее
copod = COPOD(contamination=contamination)
# Обучаем модель на обучающих данных X_train
copod.fit(X_train)
# Вызываем функцию metrics, которая вычисляет и выводит различные метрики качества модели, такие как точность, полнота, F1-мера и AUC
# Функция принимает два аргумента: модель и истинные метки y_train
metrics(copod, y_train)

**Вывод** посмотрим как модель COPOD справилась с обнаружением аномалий на обучающих данных.

- ROC: Это кривая, которая показывает, как меняется соотношение между долей верно классифицированных аномалий (True Positive Rate) и долей неверно классифицированных нормальных точек (False Positive Rate) при изменении порога для определения аномалий. Чем выше ROC, тем лучше модель разделяет аномалии и нормальные точки. Модель имеет ROC 0.6988, что означает, что она лучше справляется с этой задачей, чем модель ECOD.
- Precision: это доля точек данных, которые модель правильно определила как аномальные, среди всех точек, которые модель определила как аномальные. Чем выше precision, тем меньше ложных срабатываний модели. Модель имеет precision 0.34, что означает, что из всех точек, которые модель пометила как аномальные, 34% действительно являются аномальными. Это немного лучше, чем у модели ECOD.
- Recall: это доля точек данных, которые модель правильно определила как аномальные, среди всех аномальных точек в данных. Чем выше recall, тем меньше пропущенных аномалий моделью. Модель имеет recall 0.40, что означает, что из всех аномальных точек в данных, модель нашла 40%. Это намного лучше, чем у модели ECOD.
- F1-score: это среднее гармоническое между precision и recall, которое учитывает их баланс. Чем выше F1-score, тем лучше модель согласовывает precision и recall. Модель имеет F1-score 0.37, что означает, что она лучше балансирует между precision и recall, чем модель ECOD.
- Confusion matrix: это таблица, которая показывает, сколько точек данных было правильно или неправильно классифицировано моделью. Ваша модель имеет confusion matrix [[227154    300]
 [   235    156]], что означает, что она правильно определила 227154 нормальных точек и 156 аномальных точек, но также ошибочно определила 300 нормальных точек как аномальные и пропустила 235 аномальных точек. Это лучше, чем у модели ECOD.

- Из этих метрик вы можете сделать вывод, что модель COPOD лучше справляется с обнаружением аномалий на обучающих данных, чем модель ECOD. Возможно, мы должны попробовать эту модель на тестовых данных или сравнить ее с другими моделями.

### ROD

ROD - это сокращение от Rotation-based Outlier Detection, что означает детектор аномалий, основанный на повороте. Это устойчивый и безпараметрический алгоритм, который не требует никаких предположений о статистическом распределении данных и работает интуитивно в трехмерном пространстве, где 3D-векторы, представляющие точки данных, поворачиваются вокруг геометрической медианы два раза против часовой стрелки с помощью формулы поворота Родрига. Результаты поворота - это параллелепипеды, чьи объемы математически анализируются как целевые функции и используются для вычисления медианного абсолютного отклонения, чтобы получить оценку выброса. Для высоких размерностей > 3 общая оценка вычисляется путем усреднения общих оценок 3D-подпространств, которые были получены в результате разложения исходного пространства данных.

In [ ]:
# задаем нормализованные данные X_norm и метки y, которые хотим разделить

# вызываем функцию get_data_part с аргументами X_norm, y и size
# X_norm - это двумерный массив с данными
# y - это одномерный массив с метками, где 0 - нормальное значение, а 1 - аномальное
# size - это доля данных, которая будет использована для обучающей выборки
# функция возвращает три значения: X_train, y_train и contamination
# X_train - это двумерный массив с обучающими данными
# y_train - это одномерный массив с обучающими метками
# contamination - это доля аномалий в обучающей выборке
X_train, y_train, contamination = get_data_part(X_norm, y, size=0.3)

- Размер выборки 85442
- доля аномаоий 0.002

In [ ]:
# %%time
# rod = ROD(contamination=contamination)
# rod.fit(X_train)
# metrics(rod, y_train)

**Внимание: ниже код работает очень долго.**

In [ ]:
#%%time
# Используем магическую команду %%time для измерения времени выполнения ячейки

# Создаем объект класса ROD, который использует алгоритм обнаружения аномалий на основе метода, основанного на повороте
# contamination - это доля аномалий в данных, которая задается заранее
#rod = ROD(contamination=contamination)
# Обучаем модель на обучающих данных X_train
#rod.fit(X_train)
# Вызываем функцию metrics, которая вычисляет и выводит различные метрики качества модели, такие как точность, полнота, F1-мера и AUC
# Функция принимает два аргумента: модель и истинные метки y_train
#metrics(rod, y_train)

ROD работает слишком долго даже на 30% данных...

### Deep SVD

Deep SVD - это сокращение от Deep Singular Value Decomposition, что означает глубокое сингулярное разложение. Это метод для построения нейронной сети, которая может аппроксимировать матрицу рейтингов пользователей и объектов с помощью трех матриц: U, Σ и V^T. Этот метод основан на классическом алгоритме сингулярного разложения (SVD), который разбивает матрицу на три матрицы, такие что A = U Σ V^T, где U и V - это ортогональные матрицы, а Σ - это диагональная матрица с сингулярными значениями. Deep SVD использует нейронную сеть для обучения матриц U и V, а также для оптимизации сингулярных значений в Σ. Это позволяет учитывать нелинейные зависимости между пользователями и объектами, а также уменьшать размерность пространства признаков. Deep SVD может быть использован для решения задачи рекомендательных систем, то есть предсказания рейтингов пользователей для объектов, которые они еще не оценили.
![%D0%A1%D0%B8%D0%BD%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%BD%D0%BE%D0%B5%20%D1%80%D0%B0%D0%B7%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5.jpeg](attachment:%D0%A1%D0%B8%D0%BD%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%BD%D0%BE%D0%B5%20%D1%80%D0%B0%D0%B7%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5.jpeg)![image.png](attachment:image.png)

In [ ]:
%pip install tensorflow
# обычная установка

In [ ]:
# pip install --upgrade tensorflow # или обновление, если нужно

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
# импортируем модуль pyod.models.deep_svdd, который содержит класс DeepSVDD для обнаружения аномалий с помощью
# глубокого одноклассового метода
# DeepSVDD - это сокращение от Deep Support Vector Data Description, что означает глубокое описание данных
# с помощью опорных векторов
# Это метод для обучения нейронной сети, которая отображает данные в пространство, где они ограничены гиперсферой
# с центром c и радиусом R
# Метод минимизирует объем гиперсферы, заставляя сеть извлекать общие факторы вариации в данных
# Метод может быть использован для обнаружения аномалий, вычисляя расстояние от центра гиперсферы до каждой точки данных
# Чем больше расстояние, тем больше вероятность, что точка данных является аномальной

from pyod.models.deep_svdd import DeepSVDD

In [ ]:
# задаем нормализованные данные X_norm и метки y, которые хотим разделить

# вызываем функцию get_data_part с аргументами X_norm, y и size
# X_norm - это двумерный массив с данными
# y - это одномерный массив с метками, где 0 - нормальное значение, а 1 - аномальное
# size - это доля данных, которая будет использована для обучающей выборки
# функция возвращает три значения: X_train, y_train и contamination
# X_train - это двумерный массив с обучающими данными
# y_train - это одномерный массив с обучающими метками
# contamination - это доля аномалий в обучающей выборке

X_train, y_train, contamination = get_data_part(X_norm, y, size=0.1)

- Размер выборки 28480
- доля аномаоий 0.002

In [ ]:
# определяем функцию цели для оптимизации гиперпараметров с помощью optuna
def objective(trial):
    # получаем количество нейронов во входном слое по размеру обучающих данных
    neurons = X_train.shape[1]
    # создаем объект класса DeepSVDD, который использует алгоритм обнаружения аномалий на основе глубокого одноклассового метода
    # random_state - это параметр для воспроизводимости результатов
    # contamination - это доля аномалий в данных, которая задается заранее
    # verbose - это параметр для управления выводом информации во время обучения
    # hidden_neurons - это список, содержащий количество нейронов в скрытых слоях сети
    # use_ae - это булевый параметр, который определяет, использовать ли автоэнкодер для инициализации весов сети
    # epochs - это количество эпох для обучения сети
    # dropout_rate - это параметр для регуляризации сети, который определяет долю нейронов, которые отключаются во время обучения
    # l2_regularizer - это параметр для регуляризации сети, который определяет коэффициент L2-штрафа для весов сети
    # все эти параметры, кроме random_state и contamination, являются гиперпараметрами, которые мы хотим оптимизировать с помощью optuna
    # для этого мы используем различные методы trial.suggest_*, которые позволяют выбирать значения из заданного диапазона или списка
    # например, trial.suggest_categorical выбирает значение из списка категориальных значений
    # trial.suggest_int выбирает целочисленное значение из заданного диапазона с заданным шагом
    # trial.suggest_float выбирает вещественное значение из заданного диапазона с возможностью использовать логарифмическую шкалу
    clf = DeepSVDD(
        random_state=0,
        contamination=contamination,
        verbose=0,
        hidden_neurons=[neurons, round(neurons / 2)],
        use_ae=trial.suggest_categorical('use_ae', [True, False]),
        epochs=trial.suggest_int('epochs', 100, 300),
        dropout_rate=trial.suggest_float('dropout', 0.05, 0.5, log=True),
        l2_regularizer=trial.suggest_float('l2', 0.05, 0.5, log=True)
    )
    # обучаем модель на обучающих данных X_train
    clf.fit(X_train)
    # получаем метки аномалий для тестовых данных X_test
    y_pred = clf.labels_
    # возвращаем среднюю точность (average precision score) как метрику качества модели
    # средняя точность - это взвешенное среднее точностей, достигнутых при различных порогах, с увеличением полноты от предыдущего порога, используемого в качестве веса
    # средняя точность лежит в диапазоне от 0 до 1, чем выше, тем лучше
    return average_precision_score(y_train, y_pred)

In [ ]:
def objective(trial):
    neurons = X_train.shape[1]
    clf = DeepSVDD(
        random_state=0,
        contamination=contamination,
        verbose=0,
        hidden_neurons=[neurons, round(neurons / 2)],
        use_ae=trial.suggest_categorical('use_ae', [True, False]),
        epochs=trial.suggest_int('epochs', 100, 300),
        dropout_rate=trial.suggest_float('dropout', 0.05, 0.5, log=True),
        l2_regularizer=trial.suggest_float('l2', 0.05, 0.5, log=True)
    )
    clf.fit(X_train)
    y_pred = clf.labels_
    return average_precision_score(y_train, y_pred)

In [ ]:
# Вызываем функцию optimize, которая принимает на вход функцию objective, которая задает целевую функцию для оптимизации,
# и параметр n, который задает количество испытаний для оптимизации
# 20 - это количество испытаний, которые мы хотим провести для оптимизации параметров модели обнаружения аномалий.
# Каждое испытание выбирает случайные значения для параметров модели из заданных диапазонов или списков с помощью
# методов suggest_int, suggest_categorical и suggest_float. Затем оно обучает модель на обучающей выборке и вычисляет
# значение целевой функции, которая равна средней точности (average precision score) на обучающей выборке.
# Средняя точность - это метрика качества обнаружения аномалий, которая вычисляется как площадь под кривой точность-полнота
# (precision-recall curve). Чем выше средняя точность, тем лучше модель обнаруживает аномалии.
# После проведения всех испытаний мы можем увидеть лучшие параметры и лучшее значение целевой функции,
# которые были найдены в процессе оптимизации, а также два графика, которые показывают важность параметров
# и параллельные координаты. Важность параметров показывает, насколько каждый параметр влияет на значение целевой функции,
# а параллельные координаты показывают, как параметры и значение целевой функции меняются в разных испытаниях.
# Эти графики помогут нам понять, какие параметры лучше всего подходят для нашей модели.

optimize(objective, 20)

Параметры могут менятся:
Best params:{'use_ae': False, 'epochs': 175, 'dropout':0.13240843290940185, 'l2': 0.09164054973288507}
Best value:0.5786104975205142
Вот наши оптимальные параметры для модели Deep SVD, которая является одним из алгоритмов библиотеки PyOD.
Пояснения:
- use_ae - это булевый параметр, который определяет, использовать ли автоэнкодер для инициализации весов сети. Автоэнкодер - это нейронная сеть, которая учится сжимать и восстанавливать данные. Если use_ae равен False, то веса сети инициализируются случайным образом. В вашем случае use_ae равен False, что означает, что мы не используем автоэнкодер.
- epochs - это количество эпох для обучения сети. Эпоха - это один проход по всем обучающим данным. Чем больше эпох, тем больше возможность сети научиться выделять аномалии. Однако слишком большое количество эпох может привести к переобучению, когда сеть запоминает обучающие данные и теряет способность обобщать на новых данных. В вашем случае epochs равен 175, что означает, что мы обучаем сеть 175 раз на обучающих данных.
- dropout_rate - это параметр для регуляризации сети, который определяет долю нейронов, которые отключаются во время обучения. Регуляризация - это способ предотвратить переобучение, когда сеть становится слишком сложной и подстраивается под шум в данных. Dropout - это один из методов регуляризации, который случайным образом исключает некоторые нейроны из сети, заставляя ее учиться более устойчивым и разнообразным представлениям данных. В вашем случае dropout_rate равен 0.13, что означает, что мы отключим примерно 13% нейронов в каждом слое сети во время обучения.
- l2_regularizer - это параметр для регуляризации сети, который определяет коэффициент L2-штрафа для весов сети. L2-штраф - это еще один метод регуляризации, который добавляет к функции потерь сети дополнительный член, пропорциональный квадрату нормы весов сети. Это заставляет сеть уменьшать величину весов и предотвращать их слишком большое возрастание. В вашем случае l2_regularizer равен 0.09, что означает, что вы накладываете средний L2-штраф на веса сети.

- Best value - это метрика качества модели, которую мы выбралиранееи для оптимизации параметров. В вашем случае использована средняя точность (average precision score) как метрика качества модели.
Средняя точность - это взвешенное среднее точностей, достигнутых при различных порогах, с увеличением полноты от предыдущего порога, используемого в качестве веса. Средняя точность лежит в диапазоне от 0 до 1, чем выше, тем лучше. В нашем случае best value равен 0.57, что означает, что мы достигли средней точности около 50%.

**Вывод**

У меня возник вопрос - какие параметры лучше для обнаружения выбросов в данных:
Deep SVD, ECOD или COPOD. Я думаю, это зависит от многих факторов, таких как тип и размер данных, степень зашумленности, желаемая скорость и точность и т.д.
Я только могу предположить, некоторые критерии для сравнения этих алгоритмов.

- Deep SVD - это алгоритм, основанный на глубоком одноклассовом методе опорных векторов, который использует нейронную сеть для обучения и предсказания. Он требует библиотеку TensorFlow и имеет много гиперпараметров, которые нужно настраивать. Он может давать хорошие результаты на сложных и высокоразмерных данных, но также может быть подвержен переобучению и требовать больше времени и ресурсов для обучения и инференса.
Инференс - это процесс применения модели искусственного интеллекта (ИИ) к новым данным для получения предсказаний или решений.  Инференс отличается от обучения модели ИИ, которое заключается в настройке параметров модели на основе обучающих данных. Инференс обычно требует меньше вычислительных ресурсов, чем обучение, но все же может быть сложной и затратной задачей, особенно для больших и сложных моделей ИИ.
- ECOD - это ансамбль для обнаружения выбросов на основе кластеризации, который использует несколько базовых детекторов, таких как K-Means, DBSCAN и OPTICS, для разбиения данных на кластеры и вычисления степени аномальности каждого кластера. Он не требует библиотеку TensorFlow и имеет меньше гиперпараметров, которые нужно настраивать. Он может давать хорошие результаты на средних и низкоразмерных данных, но также может быть чувствителен к выбору базовых детекторов и параметров кластеризации.
- COPOD - это корреляционный метод обнаружения выбросов, который использует статистические свойства данных, такие как эмпирическая копула и скошенность, для оценки вероятности попадания каждого наблюдения в хвосты распределения. Он не требует библиотеку TensorFlow и не имеет гиперпараметров, которые нужно настраивать. Он может давать хорошие результаты на больших и разнородных данных, но также может быть нестабилен при наличии сильных выбросов или неоднородных распределений.

В зависимости от наших целей и ограничений, мы можете выбрать тот алгоритм, который лучше подходит для задачи.

Мы также можем сравнить их по различным метрикам качества, таким как средняя точность, ROC-кривая, F1-мера.

Спасибо за внимание!